In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **Imported Libraries**

### **1. Data Handling and Visualization**
- **`numpy`**: For numerical computations.
- **`os`**: For file and directory operations.
- **`matplotlib.pyplot`**: For plotting graphs and visualizations.
- **`seaborn`**: For creating enhanced data visualizations like heatmaps.

### **2. Deep Learning Frameworks**
- **TensorFlow/Keras**:
  - **`ImageDataGenerator`**: For augmenting and preprocessing images.
  - **`MobileNet`**: A pre-trained MobileNet model for transfer learning.
  - **`Dense`, `Dropout`, `Sequential`**: For building custom neural network layers.
  - **`Adamax`**: An optimizer for training the model.
  - **Callbacks**:
    - `EarlyStopping`: Stops training when performance stops improving.
    - `ModelCheckpoint`: Saves the model at checkpoints during training.

- **PyTorch**:
  - **`torch`** and **`torch.nn`**: For creating and training neural networks.
  - **`torchvision.transforms`** and **`models`**: For preprocessing and using pre-trained models.

### **3. Image Processing**
- **`PIL.Image`**: For handling image operations in PyTorch.
- **`load_img`, `img_to_array`**: For loading and converting images in TensorFlow/Keras.

### **4. Evaluation Metrics**
- **`classification_report`, `confusion_matrix`**: For analyzing model performance.
- **`roc_curve`, `auc`**: For plotting the Receiver Operating Characteristic (ROC) curve and calculating Area Under Curve (AUC).
- **`ConfusionMatrixDisplay`**: For visualizing confusion matrices.

---

## **Functionality**

### **1. Directory Listing**
- **`os.listdir('/kaggle/input/')`**:
  - Lists files and folders in the `/kaggle/input/` directory, which is typically used for accessing datasets in a Kaggle notebook.

### **2. Model Setup**
- Leverages **`MobileNet`** from TensorFlow for transfer learning to create a powerful image classification model.
- Includes preparation for using PyTorch as an alternative framework for building and training models.

### **3. Metrics and Visualizations**
- Implements confusion matrix, classification report, and ROC curve for a detailed evaluation of the model's performance.

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
os.listdir('/kaggle/input/')

# Explanation of the Data Preprocessing and Generators

Setting up data preprocessing and generators to prepare the dataset for training, validation, and testing using **Keras' ImageDataGenerator**. Here's a breakdown of the steps:

---

## **1. Image Dimensions**
- **`IMG_HEIGHT, IMG_WIDTH = 224, 224`**:
  - Sets the target dimensions to which all images will be resized. This is commonly used for pre-trained models like MobileNet.

---

## **2. Data Augmentation**
- **`ImageDataGenerator`**:
  - A utility for real-time data augmentation during training to improve model generalization.

### **Train Data Generator (`train_datagen`)**
- **Rescaling**: Normalizes pixel values to the range `[0, 1]` by dividing by 255.
- **Augmentations**:
  - `rotation_range=20`: Randomly rotates images within a range of 20 degrees.
  - `width_shift_range=0.25`, `height_shift_range=0.25`: Shifts the image horizontally and vertically by up to 25%.
  - `zoom_range=0.2`: Randomly zooms images in and out by 20%.
  - `horizontal_flip=True`, `vertical_flip=True`: Flips images horizontally and vertically.
  - `fill_mode='nearest'`: Fills empty pixels caused by augmentations using the nearest pixel value.
- **Validation Split**: Splits the dataset into 80% training and 20% validation.

### **Validation and Test Data Generators**
- **Validation (`val_datagen`)** and **Test (`test_datagen`)**:
  - Rescales images to normalize pixel values but do not apply augmentations since these datasets are used for evaluation.

---

## **3. Data Generators**
- **`flow_from_directory`**:
  - Loads images directly from directories and applies transformations on-the-fly.

### **Training Generator (`train_generator`)**
- Directory: `/kaggle/input/ml-project-dataset/ml_project/train/train`
- **Parameters**:
  - `target_size=(224, 224)`: Resizes images to match the dimensions required by the model.
  - `batch_size=32`: Specifies the number of images per batch.
  - `class_mode='categorical'`: Indicates multi-class classification.
  - `subset='training'`: Uses 80% of the dataset as training data.

### **Validation Generator (`val_generator`)**
- Directory: Same as training (`/kaggle/input/ml-project-dataset/ml_project/train/train`).
- **Parameters**:
  - Same as `train_generator` but uses `subset='validation'` to select the validation split.
  - `shuffle=True`: Ensures the validation data is shuffled.

### **Test Generator (`test_generator`)**
- Directory: `/kaggle/input/ml-project-dataset/ml_project/test/test`
- **Parameters**:
  - `shuffle=False`: Ensures the test data is not shuffled for consistent evaluation.

---

## **Summary**
- Preprocesses and augments training data for better model generalization.
- Sets up separate generators for training, validation, and testing.
- Ensures the data is correctly split and preprocessed for a smooth training pipeline.


In [ ]:
IMG_HEIGHT , IMG_WIDTH = 224, 224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split = 0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/lost-and-found-project-dataset/ml_project/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical',
    subset = 'training'
)

val_generator = train_datagen.flow_from_directory(
    '/kaggle/input/lost-and-found-project-dataset/ml_project/train',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical',
    subset = 'validation',
    shuffle = True
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/lost-and-found-project-dataset/ml_project/test',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Explanation of the Model Definition

Defining a deep learning model for multi-class image classification using **MobileNet** as the base model and additional custom layers for classification.

---

## **1. Number of Classes**
- **`num_classes = 11`**:
  - Indicates the number of output classes in the classification task.

---

## **2. Base Model**
- **`base_model = MobileNet(...)`**:
  - **MobileNet**: A pre-trained model loaded with `ImageNet` weights.
  - **Parameters**:
    - `weights='imagenet'`: Uses pre-trained weights from the ImageNet dataset.
    - `include_top=False`: Removes the top classification layer to add custom layers.
    - `pooling='avg'`: Applies global average pooling to the feature maps.
    - `input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)`: Specifies the input image dimensions.

- **Freezing Layers**:
  - **`for layer in base_model.layers[:-10]: layer.trainable = False`**:
    - Freezes all but the last 10 layers of MobileNet to prevent overfitting while fine-tuning the model on the new dataset.

---

## **3. Custom Model Architecture**
- **`Sequential()`**: A sequential model is used to stack layers linearly.

### Added Layers:
1. **Base Model**: MobileNet as the feature extractor.
2. **Dense Layer (256)**:
   - Adds a fully connected layer with 256 neurons and ReLU activation.
   - **Dropout (0.45)**: Reduces overfitting by randomly disabling 45% of neurons during training.
3. **Dense Layer (128)**:
   - Adds another fully connected layer with 128 neurons and ReLU activation.
   - **Dropout (0.3)**: Reduces overfitting by disabling 30% of neurons.
4. **Dense Layer (64)**:
   - Fully connected layer with 64 neurons and ReLU activation.
5. **Output Layer**:
   - **`Dense(num_classes, activation='softmax')`**:
     - Outputs probabilities for each of the 11 classes using the softmax activation function.

---

## **4. Compilation**
- **`model.compile(...)`**:
  - **Optimizer**: `Adamax` with a learning rate of 0.001.
  - **Loss Function**: `categorical_crossentropy` for multi-class classification.
  - **Metrics**: Tracks `accuracy` during training and evaluation.

---

## **5. Model Check**
- **Dummy Input**:
  - **`dummy_input = np.random.random((1, IMG_HEIGHT, IMG_WIDTH, 3))`**:
    - Generates a random input tensor to ensure the model is built correctly.
  - **`model(dummy_input)`**: Verifies that the model can process the input without errors.

- **Model Summary**:
  - **`model.summary()`**:
    - Displays the architecture, trainable parameters, and output shapes.

---

## **Summary**
This code builds a transfer learning model using MobileNet as a feature extractor with additional custom dense layers for classification. It uses dropout layers to prevent overfitting and compiles the model with an Adamax optimizer and categorical cross-entropy loss for multi-class classification.


In [ ]:
num_classes = 11

base_model = MobileNet(weights='imagenet', include_top=False, pooling='avg', input_shape=(IMG_HEIGHT , IMG_WIDTH, 3))

for layer in base_model.layers[:-10]:
    layer.trainable = False

model = Sequential()

model.add(base_model)

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.45))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(64, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adamax(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

dummy_input = np.random.random((1, IMG_HEIGHT, IMG_WIDTH, 3))
model(dummy_input)

model.summary()


# Explanation of the Training Setup and Callbacks

This section defines the training process for the model using **early stopping** and **model checkpointing** to improve training efficiency and save the best-performing model.

---

## **1. Early Stopping**
- **`EarlyStopping(...)`**:
  - A callback to stop training when the validation performance stops improving.
  - **Parameters**:
    - `monitor='val_loss'`: Monitors the validation loss during training.
    - `patience=5`: Stops training if the validation loss does not improve for 5 consecutive epochs.
    - `restore_best_weights=True`: Restores the model weights to the best-performing epoch after training stops.

---

## **2. Model Checkpoint**
- **`ModelCheckpoint(...)`**:
  - A callback to save the model at checkpoints during training.
  - **Parameters**:
    - `'best_model.keras'`: Saves the model as a file named `best_model.keras`.
    - `save_best_only=True`: Ensures only the best model (with the lowest validation loss) is saved.
    - `monitor='val_loss'`: Saves the model based on validation loss.

---

## **3. Training the Model**
- **`model.fit(...)`**:
  - Trains the model using the preprocessed training and validation data.
  - **Parameters**:
    - `train_generator`: The training dataset generator with augmented images.
    - `validation_data=val_generator`: The validation dataset generator.
    - `epochs=20`: Specifies a maximum of 20 epochs for training.
    - `callbacks=[early_stopping, model_checkpoint]`: Includes the callbacks for early stopping and saving the best model.

---

## **4. History Object**
- The **`history`** object stores training details such as:
  - Training and validation loss.
  - Training and validation accuracy.
- This object can be used to visualize the training progress and performance metrics.

---

## **Summary**
This code sets up the training process with:
1. Early stopping to prevent overfitting and save training time.
2. Model checkpointing to save the best-performing model.
3. Training and validation data generators to monitor performance.


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stopping, model_checkpoint],
)

# Explanation of Model Evaluation on Test Data

This section evaluates the trained model on the test dataset to measure its performance on unseen data.

---

## **1. Loading Best Model Weights**
- **`model.load_weights('best_model.keras')`**:
  - Loads the weights of the model saved during training.
  - These weights correspond to the best-performing model based on validation loss (as monitored by the `ModelCheckpoint` callback).

---

## **2. Evaluating on Test Data**
- **`model.evaluate(test_generator)`**:
  - Evaluates the model's performance on the test dataset.
  - **Parameters**:
    - `test_generator`: The test dataset generator prepared earlier.
  - **Returns**:
    - `test_loss`: The loss on the test dataset, calculated using the loss function (`categorical_crossentropy`).
    - `test_acc`: The accuracy on the test dataset, as specified in the `metrics` parameter during compilation.

---

## **3. Printing Test Accuracy**
- **`print(f"Test accuracy: {test_acc:.2f}")`**:
  - Displays the test accuracy in a formatted manner with two decimal places.

---

## **Summary**
This code:
1. Loads the best model weights saved during training.
2. Evaluates the model on unseen test data to compute the test loss and accuracy.
3. Prints the test accuracy as a measure of the model's generalization capability.

In [ ]:
model.load_weights('best_model.keras')
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc:.2f}")

# Explanation of Model Prediction and Evaluation

This section performs predictions on the test data, converts the predictions to class labels, and evaluates the model's performance using the classification report.

---

## **1. Predicting on Test Data**
- **`y_pred = model.predict(test_generator)`**:
  - Uses the trained model to predict the class probabilities for each image in the test dataset.
  - **`test_generator`**: A generator that provides batches of test images for the model to predict.

---

## **2. Converting Predictions to Class Labels**
- **`y_pred_classes = np.argmax(y_pred, axis=1)`**:
  - **`np.argmax()`**: Converts the predicted probabilities into class labels by selecting the index of the highest probability in each prediction.
  - **`axis=1`**: Operates along the class axis, i.e., for each prediction (each image), it selects the class with the highest probability.

---

## **3. Extracting True Labels**
- **`y_true = test_generator.classes`**:
  - Extracts the true class labels from the test data generator.
  - **`test_generator.classes`**: Contains the true class labels of all images in the test set.

---

## **4. Generating Classification Report**
- **`classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())`**:
  - Generates a classification report that includes precision, recall, f1-score, and support for each class.
  - **`y_true`**: The true labels of the test set.
  - **`y_pred_classes`**: The predicted class labels.
  - **`target_names=test_generator.class_indices.keys()`**: Maps class indices to class labels, so the report displays the actual class names instead of numerical labels.

---

## **Summary**
This code:
1. Uses the trained model to predict the class probabilities for the test data.
2. Converts the predicted probabilities into class labels.
3. Generates and prints a detailed classification report that evaluates the model's performance across each class based on precision, recall, and F1-score.


In [ ]:
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_generator.classes

print(classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys()))

In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
def preprocess_query_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size) 
    image_array = img_to_array(image)                   
    image_array = np.expand_dims(image_array, axis=0)   
    image_array = image_array / 255.0                   
    return image_array

query_image_path = ""

query_image = preprocess_query_image(query_image_path, target_size=(IMG_HEIGHT, IMG_WIDTH))

predictions = model.predict(query_image)
predicted_class_index = np.argmax(predictions, axis=1)[0]

class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}
query_image_category = class_labels[predicted_class_index]

print(f"Predicted category for the query image: {query_image_category}")


# Explanation of Siamese Network Model for Image Similarity

This code defines a **Siamese Network** to find the most similar image to a given query image by comparing feature embeddings. The model uses a pre-trained ResNet-18 model for feature extraction, then performs image similarity based on cosine similarity.

---

## **1. Siamese Network Class**
- **`class SiameseNetwork(nn.Module):`**:
  - Defines the Siamese network architecture using PyTorch's `nn.Module`.
  - The network utilizes a pre-trained **ResNet-18** model, excluding the final classification layer, to extract image features.
- **`self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])`**:
  - The `base_model` is a ResNet-18 pre-trained on ImageNet.
  - We remove the final layer to only keep the feature extraction part of the model.
  
---

## **2. Forward Pass**
- **`def forward(self, x):`**:
  - Defines the forward pass for the network, which extracts features from the input image.
  - The output is reshaped using `.view()` to a flat vector of features.

---

## **3. Image Transformations**
- **`transform = transforms.Compose([...])`**:
  - Applies a series of transformations to the input image:
    - **Resize** the image to 224x224 pixels.
    - **ToTensor** converts the image to a tensor.
    - **Normalize** adjusts the pixel values based on the ImageNet mean and standard deviation.

---

## **4. Model Initialization**
- **`device = torch.device("cuda" if torch.cuda.is_available() else "cpu")`**:
  - Automatically detects whether a GPU is available and sets the device accordingly (either CUDA or CPU).
- **`model = SiameseNetwork().to(device)`**:
  - Initializes the Siamese network and moves it to the appropriate device (GPU/CPU).
- **`model.eval()`**:
  - Sets the model to evaluation mode (no gradient updates).

---

## **5. Get Image Embedding**
- **`def get_embedding(image_path, model):`**:
  - Loads an image from the specified path, applies the transformations, and computes the feature embedding using the Siamese network.
  - **`embedding = model(image_tensor).cpu().numpy()`**: The embedding is obtained by passing the image tensor through the model.

---

## **6. Finding the Most Similar Image**
- **`def find_most_similar(query_image_path, folder_path, model):`**:
  - Given a query image, this function finds the most similar image from a folder using cosine similarity between their embeddings.
  - The similarity score is computed as:
    - **`similarity = np.dot(query_embedding, folder_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(folder_embedding))`**.
  - If the similarity score is greater than 0.80, it considers that image as a potential match.

---


## **Summary**
This code performs image similarity comparison using a Siamese network with a ResNet-18 backbone. It extracts feature embeddings from images, computes the cosine similarity between the query image and others in the dataset, and identifies the most similar image.


In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        base_model = models.resnet18(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])

    def forward(self, x):
        features = self.feature_extractor(x)
        return features.view(features.size(0), -1)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork().to(device)
model.eval()

def get_embedding(image_path, model):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model(image_tensor).cpu().numpy()
    return embedding.flatten()

def find_most_similar(query_image_path, folder_path, model):
    query_embedding = get_embedding(query_image_path, model)
    max_similarity = -1
    most_similar_image = None

    for file in os.listdir(folder_path):
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, file)
            folder_embedding = get_embedding(image_path, model)
            similarity = np.dot(query_embedding, folder_embedding) / (
                np.linalg.norm(query_embedding) * np.linalg.norm(folder_embedding)
            )

            if similarity > max_similarity and similarity > 0.80:
                max_similarity = similarity
                most_similar_image = image_path

    return most_similar_image, max_similarity

if __name__ == "__main__":
    query_image = ""
    images_folder = f"{query_image_category}"
    
    similar_image, similarity_score = find_most_similar(query_image, images_folder, model)
    print(f"Most similar image: {similar_image} with similarity score: {similarity_score:.4f}")


In [ ]:
import matplotlib.pyplot as plt

def display_images_side_by_side(query_image_path, similar_image_path):
    query_image = Image.open(query_image_path).convert("RGB")
    similar_image = Image.open(similar_image_path).convert("RGB")

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    axes[0].imshow(query_image)
    axes[0].axis("off")
    axes[0].set_title("Query Image")
    
    axes[1].imshow(similar_image)
    axes[1].axis("off")
    axes[1].set_title("Most Similar Image")
    
    plt.tight_layout()
    plt.show()
    
if similar_image:
    display_images_side_by_side(query_image, similar_image)
else:
    print("No similar images found in the folder.")
